In [1]:
A = matrix(RDF, [[1.8, 1.2, -0.9],
                 [1.2, 1.3, 0.8],
                 [-0.9, 0.8, 0.8]])

In [2]:
def indexof_max(A, non_diag=False, func=lambda x: x):
    max_i, max_j = (0, 0)
    cur_max = -oo
    for i in range(A.nrows()):
        for j in range(A.ncols()):
            if (i == j) and (non_diag):
                continue
            a_ij = func(A[i, j])
            if a_ij > cur_max:
#                 pretty_print(html('New max: %s' % A[i, j]))
                cur_max = a_ij
                max_i = i
                max_j = j
    return (max_i, max_j)

In [3]:
def C_k(A, method=2):
    i, j = indexof_max(A, True, abs)
#     pretty_print(html('$\max |a_{ij}|=A_{%s%s}=%s$' % (i, j, A[i,j])))
    d = sqrt((A[i, i] - A[j, j])^2 + 4 * A[i, j]^2)
    c = sqrt(1/2 * (1 + abs(A[i, i] - A[j, j])/d))
    s = sign(A[i, j] * (A[i, i] - A[j, j])) * sqrt(1/2 * (1 - abs(A[i, i] - A[j, j]) / d))
    if method == 1:
        T = identity_matrix(RDF, A.nrows())
        T[i, i] = c
        T[i, j] = -s
        T[j, i] = s
        T[j, j] = c
#         show('\mathbf{T}=' + latex(T))
        C = T.T * A * T
    elif method == 2:
        C = copy(A)
        for k in range(A.nrows()):
            C[k, i] = c * A[k, i] + s * A[k ,j]
            C[i, k] = C[k, i]
            C[k, j] = -s * A[k, i] + c * A[k, j]
            C[j, k] = C[k, j]
        C[i, i] = (A[i, i] + A[j, j]) / 2 + sign(A[i, i] - A[j, j]) * d/2
        C[j, j] = (A[i, i] + A[j, j]) / 2 - sign(A[i, i] - A[j, j]) * d/2
        C[i, j] = 0
        C[j, i] = 0
    else:
        raise ValueError("Unknown method '%s'." % method)
        
#     show('\mathbf{C}=' + latex(C))
    return C

In [8]:
def eigenproblem_jacobi(A, method=1, eps=10e-6, max_iter=20):
    C = copy(A)
    prev_evals = vector(RDF, [oo for _ in range(A.nrows())])
    evals = vector(RDF, [0 for _ in range(A.nrows())])
    t = 0
    while any([abs(l) > eps for l in (evals - prev_evals)]) and t < max_iter:
#         print(t)
#         show('\mathbf{A}=' + latex(C))
        C = C_k(C, method)
        show('\mathbf{C}=' + latex(C))
        prev_evals = evals
        evals = vector(C.diagonal())
        t += 1
    return C, t

In [9]:
show(eigenproblem_jacobi(A, 1, max_iter=9))

\mathbf{C}= \left(\begin{array}{rrr}
2.77576506721 & 0.0 & -0.193572233 \\
-8.32667268469 \times 10^{-17} & 0.324234932787 & 1.18849896534 \\
-0.193572233 & 1.18849896534 & 0.8
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.77576506721 & 0.122711582228 & -0.149706636378 \\
0.122711582228 & -0.64995426845 & 1.11022302463 \times 10^{-16} \\
-0.149706636378 & 2.22044604925 \times 10^{-16} & 1.77418920124
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.79766311146 & 0.12141952283 & 1.11022302463 \times 10^{-16} \\
0.12141952283 & -0.64995426845 & 0.0177604022651 \\
0.0 & 0.0177604022651 & 1.75229115699
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.8019340204 & -1.2490009027 \times 10^{-16} & 0.000624332703158 \\
-1.73472347598 \times 10^{-16} & -0.654225177382 & 0.0177494252666 \\
0.000624332703158 & 0.0177494252666 & 1.75229115699
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.8019340204 & -4.60443271779 \times 10^{-06} & 0.000624315724159 \\
-4.60443271784 \times 10^{-06} & -0.654356082357 & 1.58206781009 \times 10^{-15} \\
0.000624315724159 & 1.70176372993 \times 10^{-15} & 1.75242206196
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.80193439178 & -4.60443190312 \times 10^{-06} & 4.74138957263 \times 10^{-14} \\
-4.60443190317 \times 10^{-06} & -0.654356082357 & 2.73900631416 \times 10^{-09} \\
4.73065597112 \times 10^{-14} & 2.73900643382 \times 10^{-09} & 1.75242169058
\end{array}\right)

(
[      2.80193439177771 -4.604431903124448e-06 4.7413895726267086e-14]
[-4.604431903172303e-06    -0.6543560823571031 2.7390063141556958e-09]
[ 4.730655971119102e-14 2.7390064338231697e-09     1.7524216905793943],

6
)

In [10]:
show(eigenproblem_jacobi(A, 2, max_iter=9))

\mathbf{C}= \left(\begin{array}{rrr}
2.77576506721 & 0.0 & -0.193572233 \\
0.0 & 0.324234932787 & 1.18849896534 \\
-0.193572233 & 1.18849896534 & 0.8
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.77576506721 & 0.122711582228 & -0.149706636378 \\
0.122711582228 & -0.64995426845 & 0.0 \\
-0.149706636378 & 0.0 & 1.77418920124
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.79766311146 & 0.12141952283 & 0.0 \\
0.12141952283 & -0.64995426845 & 0.0177604022651 \\
0.0 & 0.0177604022651 & 1.75229115699
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.8019340204 & 0.0 & 0.000624332703158 \\
0.0 & -0.654225177382 & 0.0177494252666 \\
0.000624332703158 & 0.0177494252666 & 1.75229115699
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.8019340204 & -4.60443271767 \times 10^{-06} & 0.000624315724159 \\
-4.60443271767 \times 10^{-06} & -0.654356082357 & 0.0 \\
0.000624315724159 & 0.0 & 1.75242206196
\end{array}\right)

\mathbf{C}= \left(\begin{array}{rrr}
2.80193439178 & -4.604431903 \times 10^{-06} & 0.0 \\
-4.604431903 \times 10^{-06} & -0.654356082357 & 2.73900473198 \times 10^{-09} \\
0.0 & 2.73900473198 \times 10^{-09} & 1.75242169058
\end{array}\right)

(
[    2.8019343917777095  -4.60443190299972e-06                    0.0]
[ -4.60443190299972e-06    -0.6543560823571031 2.7390047319849426e-09]
[                   0.0 2.7390047319849426e-09     1.7524216905793941],

6
)

In [7]:
A.eigenvalues()

[-0.6543560823632375, 2.8019343917838446, 1.752421690579394]